In [18]:
import pandas as pd 
import numpy as np 
import torch
import matplotlib.pyplot as plt 
from PIL import Image
import os
from dotenv import load_dotenv

In [19]:
model_1=torch.hub.load("ultralytics/yolov5",'custom',path='best_yolov5_weights.pt',device=torch.device('mps'))

Using cache found in /Users/smudge/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-9-28 Python-3.11.4 torch-2.1.0 MPS

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [20]:
#set the env variables
load_dotenv('./.env')
PATH_TO_TEST=os.getenv("PATH_TO_TEST")

In [21]:
#Get the image ids
test_ids=os.listdir(PATH_TO_TEST)

In [22]:

def get_predictions(img_names,path,aug,model):

    model.conf=0.30

    pred_scores=dict()
    pred_boxes=dict()
    pred_classes=dict()

    for img in img_names:
        if '_' not in img:
            img_path=path+img

            prediction=model(img_path,size=1024,augment=aug)

            pred_scores[img]=list(prediction.xyxy[0][:,4].to(device=torch.device('cpu')).detach().numpy())
            pred_boxes[img]=list(prediction.xyxy[0][:,0:4].to(device=torch.device('cpu')).detach().numpy())
            pred_classes[img]=list(prediction.xyxy[0][:,5].to(device=torch.device('cpu')).detach().numpy())
    
    return pred_scores,pred_boxes,pred_classes

In [23]:
#get the predictions
scores_dict,boxes_dict,_=get_predictions(test_ids,PATH_TO_TEST,aug=False,model=model_1)

In [24]:
boxes=list()
ids=list()
scores=list()

for i in boxes_dict.keys():
    boxes.append((boxes_dict[i]))
    ids.append(i)
    scores.append((scores_dict[i]))

df=pd.DataFrame({
    'image_id':ids,
    'boxes':boxes,
    'scores':scores
})

df.to_csv("./Results/test_conf_0.30.csv")